In [ ]:
#@title **set up**

# install mediapipe
!pip install mediapipe

# clone github code
!git clone https://github.com/cedro3/mediapipe.git
%cd mediapipe/

# initial setting
import mediapipe as mp
mp_holistic = mp.solutions.holistic

# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.5)

# Prepare DrawingSpec for drawing the face landmarks later.
WHITE_COLOR = (224, 224, 224)
BLACK_COLOR = (0, 0, 0)
RED_COLOR = (0, 0, 255)
GREEN_COLOR = (0, 128, 0)
BLUE_COLOR = (255, 0, 0)
mp_drawing = mp.solutions.drawing_utils 
drawing_face_spec = mp_drawing.DrawingSpec(color=WHITE_COLOR, thickness=1, circle_radius=1)
drawing_pose_spec = mp_drawing.DrawingSpec(color=WHITE_COLOR, thickness=3, circle_radius=3)
drawing_hand_spec = mp_drawing.DrawingSpec(color=WHITE_COLOR, thickness=3, circle_radius=3)
drawing_dot_spec = mp_drawing.DrawingSpec(color=RED_COLOR, thickness=2, circle_radius=3)

# define fuction
!pip install function
from function import *

#諸々モジュール
!pip install opencv-python
import cv2
!pip install function
from function import *
!pip install mecab-python3

#画像読み込みのセットアップ
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.1.4 which is incompatible.
Cloning into 'mediapipe'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 162 (delta 62), reused 49 (delta 49), pack-reused 84
Receiving objects: 100% (162/162), 2.40 MiB | 22.12 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/mediapipe
Looking in indexes: h

In [2]:
#@title **video-to-frames**
#@markdown upload video(mp4) with sound to movie/video folder

#諸々モジュール
!pip install opencv-python
import cv2
print( cv2.__version__ )

#・Google Cloud SDKの認証
#authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

#id = 'https://drive.google.com/file/d/1dPtsvCAnthoiSXNJDRESSiOjLJM9TcQV/view?usp=share_link'
#video= drive.CreateFile({'id': id})

video = 'ishihara.mp4' #@param {type:"string"}
video_file = 'video/'+video
image_dir='frames/'
image_file='%s.jpg'

def video_2_images(video_file= video_file,   # ビデオの指定
                   image_dir='images/', 
                   image_file='%s.jpg'):     
    # Initial setting
    i = 0
    interval = 1
    length = 3000  # 最大フレーム数
    
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)  # fps取得
 
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()
    return fps, i, interval


import os
import shutil
 
def reset_folder(path):
    if os.path.isdir(path):
      shutil.rmtree(path)
    os.makedirs(path,exist_ok=True)
 

# video_2_images
reset_folder('frames')
fps, i, interval = video_2_images(video_file, image_dir, image_file)
 
# display strat frame
from google.colab.patches import cv2_imshow
img = cv2.imread('frames/000000.jpg')
cv2_imshow(img)
 
# display parameter
print('fps = ', fps)
print('frames = ', i)
print('interval = ', interval)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
4.6.0


AttributeError: ignored

In [ ]:
#@title **MediaPipe from frames to images**

import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
from tqdm import tqdm ###

# reset images folder
reset_folder('images')

# image file names to files in list format
files=[]
for name in sorted(glob.glob('./frames/*.jpg')):
    files.append(name)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in files}

#for name, image in images.items():
for name, image in tqdm(images.items()):  
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

  # Draw pose landmarks.
  annotated_image = image.copy()
  mp_drawing.draw_landmarks(
      image=annotated_image,
      landmark_list=results.left_hand_landmarks,
      connections=mp_holistic.HAND_CONNECTIONS,
      landmark_drawing_spec=drawing_dot_spec,
      connection_drawing_spec=drawing_hand_spec)
  mp_drawing.draw_landmarks(
      image=annotated_image,
      landmark_list=results.right_hand_landmarks,
      connections=mp_holistic.HAND_CONNECTIONS,
      landmark_drawing_spec=drawing_dot_spec,
      connection_drawing_spec=drawing_hand_spec)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.face_landmarks, 
      connections=mp_holistic.FACEMESH_TESSELATION,  ###
      landmark_drawing_spec=drawing_face_spec,
      connection_drawing_spec=drawing_face_spec)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.pose_landmarks, 
      connections=mp_holistic.POSE_CONNECTIONS,
      landmark_drawing_spec=drawing_pose_spec,
      connection_drawing_spec=drawing_pose_spec)
  
  save_name = 'images/'+os.path.basename(name) ###
  cv2.imwrite(save_name, annotated_image)  

In [ ]:
#@title **make movie from images**
#@markdown　check with_sound if the video has sound
with_sound = True #@param {type:"boolean"}
fps_r = fps/interval

print('making movie...')
if with_sound ==  True:  
  ! ffmpeg -y -r $fps_r -i images/%6d.jpg -vcodec libx264 -pix_fmt yuv420p -loglevel error out.mp4
  # audio extraction/addition
  print('preparation for sound...')
  ! ffmpeg -y -i $video_file -loglevel error sound.mp3
  ! ffmpeg -y -i out.mp4 -i sound.mp3 -loglevel error output.mp4
else:
  ! ffmpeg -y -r $fps_r -i images/%6d.jpg -vcodec libx264 -pix_fmt yuv420p -loglevel error output.mp4

display_mp4('output.mp4')

In [ ]:
#@title **download movie** (chrome)
from google.colab import files
files.download('output.mp4')